In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sencsv/sen.csv


In [2]:
import torch
from torch import nn  #nn.Module是PyTorch提供的神经网络类，并在类中实现了网络各层的定义及前向计算与反向传播机制
import torch.nn.functional as F   #包含 torch.nn 库中所有函数同时包含大量 loss 和 activation function
from tqdm.notebook import tqdm   #tqdm中的tqdm()是实现进度条美化的基本方法
from sklearn.metrics import classification_report   #显示分类指标
from transformers import BertTokenizer  #BertTokenizer分词器
from transformers import BertModel  #对象构建
from torch.utils.data import TensorDataset #TensorDataset(tensor1,tensor2)的功能就是形成数据tensor1和标签tensor2的对应

In [3]:
# 提取关键词后的数据
df = pd.read_csv('/kaggle/input/sencsv/sen.csv',encoding='gbk',usecols = ['title','text']) #读取数据csv文件
columns = ['text','category'] #定义表格文件列标题
df.columns = columns #定义表格文件标题

In [4]:
df.dropna(axis=0, how='any', inplace=True) #drop掉所有可能出现nan的行并覆盖原有的df
df[df.isnull().T.any().T]  #找出含有nan的所有行
df.reset_index(drop=True,inplace=True) #重置索引值并覆盖原有的df

In [5]:
df.category.value_counts() #统计数类别个数

体育    5000
娱乐    5000
时政    5000
科技    5000
教育    4999
房产    4998
财经    4998
游戏    4996
时尚    4992
家居    4980
Name: category, dtype: int64

In [6]:
possible_labels = df.category.unique() #获取不重复的唯一的category
possible_labels #查看

array(['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'],
      dtype=object)

In [7]:
#标签对应相应的数字
label_dict = {}  #定义一个新的字典
for index, possible_label in enumerate(possible_labels):  #循环查看possible_labels以及对应的索引值
    label_dict[possible_label] = index #让字典中的key（possible_label）与其索引值一一对应
label_dict   #查看

{'体育': 0,
 '娱乐': 1,
 '家居': 2,
 '房产': 3,
 '教育': 4,
 '时尚': 5,
 '时政': 6,
 '游戏': 7,
 '科技': 8,
 '财经': 9}

In [8]:
df.category = df['category'].map(label_dict) #map做映射，把category 映射成number

In [9]:
df.head()

,text,category
0,训练沈阳没有感冒大雨球员队员长春雨水受伤控制热身赛队伍出现全队球队受到青睐抵达开始干扰迹象情...,0
1,瑞典美国首战战术中国姑娘击败比赛出来逼抢没有过程进行成为软肋记者主帅特点分解形成发动占据对手...,0
2,活动提供派对冠军游戏俱乐部女士都市眼镜球队特等奖打位运动员训练迎新区域社会文静调整机会滚进面...,0
3,俱乐部球员工资注册危机奖金球队缺乏签约管理集团离开问题没有时候经验支付前提方面员工教练法律拖...,0
4,总局警方要求电话带走立案侦查没有会议发出接到涉案辽宁足球贿赂外界配合来自公安部门声明产业依法...,0


In [10]:
from sklearn.model_selection import train_test_split #在数据集上随机划分出一定比例的训练集和测试集
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.category.values, 
                                                  test_size=0.1, 
                                                  random_state=42,
                                                  stratify=df.category.values)
#df.index.values：被划分的样本特征集。
#df.category.values：被划分的样本标签集。
#使用train_test_split拆分 训练集和测试集90%为训练集，随机数42

In [11]:
df['data_type'] = ['not_set']*df.shape[0]  #新建一列data_type

In [12]:
df.head()

,text,category,data_type
0,训练沈阳没有感冒大雨球员队员长春雨水受伤控制热身赛队伍出现全队球队受到青睐抵达开始干扰迹象情...,0,not_set
1,瑞典美国首战战术中国姑娘击败比赛出来逼抢没有过程进行成为软肋记者主帅特点分解形成发动占据对手...,0,not_set
2,活动提供派对冠军游戏俱乐部女士都市眼镜球队特等奖打位运动员训练迎新区域社会文静调整机会滚进面...,0,not_set
3,俱乐部球员工资注册危机奖金球队缺乏签约管理集团离开问题没有时候经验支付前提方面员工教练法律拖...,0,not_set
4,总局警方要求电话带走立案侦查没有会议发出接到涉案辽宁足球贿赂外界配合来自公安部门声明产业依法...,0,not_set


In [13]:
df.loc[X_train, 'data_type'] = 'train' #赋值data_type为训练集train
df.loc[X_val, 'data_type'] = 'val' #赋值data_type为测试集val

In [14]:
df.groupby(['category', 'data_type']).count()  #进行数据的分组统计类别，数据标签分布

text
category data_type      
0        train      4500
         val         500
1        train      4500
         val         500
2        train      4482
         val         498
3        train      4498
         val         500
4        train      4499
         val         500
5        train      4493
         val         499
6        train      4500
         val         500
7        train      4496
         val         500
8        train      4500
         val         500
9        train      4498
         val         500

In [15]:
from transformers import BertTokenizer #导入BertTokenizer分词器
from torch.utils.data import TensorDataset

In [16]:
# 中文分词
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-chinese'
)

In [17]:
max_length=256 #句子最大长度256

# 以batch形式编码句子,输入为句子的列表
encoded_data_train = tokenizer.batch_encode_plus(  #采用bert 分词器进行分词（训练集）
    df[df.data_type=='train'].text.values,  #文本值
    add_special_tokens=True,  #序列加上特殊符号，如[CLS],[SEP]
    return_attention_mask=True,  #返回
    padding='max_length', #补填到最大句子长度
    truncation=True,  #截断操作，超过则截取
)

encoded_data_val = tokenizer.batch_encode_plus(  #采用bert 分词器进行分词（测试集）
    df[df.data_type=='val'].text.values, #文本值
    add_special_tokens=True,  #序列加上特殊符号，如[CLS],[SEP]
    return_attention_mask=True,
    padding='max_length', #补填到最大句子长度
    truncation=True,   #截断操作，超过则截取
)

input_ids_train = encoded_data_train['input_ids']  #获取训练集bert输入（input_ids）
attention_masks_train = encoded_data_train['attention_mask']  #获取bert输入（attention_mask）
labels_train = torch.tensor(df[df.data_type=='train'].category.values) #获取label 并转化成tensor

input_ids_val = encoded_data_val['input_ids']  #获取测试集bert输入（input_ids）
attention_masks_val = encoded_data_val['attention_mask'] #获取bert输入（attention_mask）
labels_val = torch.tensor(df[df.data_type=='val'].category.values) #获取label 并转化成tensor

In [18]:
df[df.data_type=='train'] #查看训练数据

,text,category,data_type
0,训练沈阳没有感冒大雨球员队员长春雨水受伤控制热身赛队伍出现全队球队受到青睐抵达开始干扰迹象情...,0,train
1,瑞典美国首战战术中国姑娘击败比赛出来逼抢没有过程进行成为软肋记者主帅特点分解形成发动占据对手...,0,train
2,活动提供派对冠军游戏俱乐部女士都市眼镜球队特等奖打位运动员训练迎新区域社会文静调整机会滚进面...,0,train
3,俱乐部球员工资注册危机奖金球队缺乏签约管理集团离开问题没有时候经验支付前提方面员工教练法律拖...,0,train
5,比赛鹿队联赛出现表现不能队员北京阿的江大家外援机会没有开赛回到希望命中率来讲状态成为点评压力...,0,train
...,...,...,...
49957,基金分配分红收益低于实现女士进行收益分配价值比例年度投资数据不得精选问题条件大盘份额牛市符合...,9,train
49958,投资配置资产基金风险产品决定组合情况股票对象比例股票投资类型资金影响个人整体高风险与否结果指...,9,train
49959,基金股票投资公司业绩市场收益仓位旗下个股受益显示中国能源大盘操作金融型基金政策数据华夏加仓总...,9,train
49961,上涨基金交易全线开盘接近基指午盘上扬封闭式涨幅放大交易日呈现A股市场格局融通收复提振大关招商...,9,train


In [19]:
input_ids_train = torch.tensor(input_ids_train)  #训练集输入转换成tensor
input_ids_val = torch.tensor(input_ids_val) #验证集输入转换成tensor
attention_masks_train = torch.tensor(attention_masks_train) #训练集attention_masks转换成tensor
attention_masks_val = torch.tensor(attention_masks_val) #验证集attention_masks转换成tensor

dataset_train = TensorDataset(input_ids_train,   #采用TensorDataset包封装成训练集
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,  #采用TensorDataset包封装成测试集
                            attention_masks_val,
                           labels_val)

In [20]:
len(dataset_train) #查看训练集长度

44966

In [21]:
dataset_train.tensors  #查看训练集

(tensor([[ 101, 6378, 5298,  ...,    0,    0,    0],
         [ 101, 4448, 1073,  ...,    0,    0,    0],
         [ 101, 3833, 1220,  ...,    0,    0,    0],
         ...,
         [ 101, 1825, 7032,  ...,    0,    0,    0],
         [ 101,  677, 3885,  ...,    0,    0,    0],
         [ 101, 1825, 7032,  ...,    0,    0,    0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 0, 0,  ..., 9, 9, 9]))

In [22]:
class BertLSTM(nn.Module):  #定义BertLSTM类继承nn.Module
    """
    Bert+TextCNN2D
    """

    def __init__(self, no_layers,hidden_dim,embedding_dim,drop_prob=0.5,
                 trained=True):  #定义初始化参数

        super(BertLSTM, self).__init__()  #子类继承父类固定写法

        self.bert = BertModel.from_pretrained('bert-base-chinese')  #定义预训练BertModel
        for param in self.bert.parameters():  
            param.requires_grad = trained  #对bert进行参数更新

        self.output_dim = output_dim  #定义输出维度
        self.hidden_dim = hidden_dim  #定义隐藏层维度
 
        self.no_layers = no_layers  #定义LSTM层数
        
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True, bidirectional=True)  #定义lstm层
        
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
    
        # linear and sigmoid layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
#         self.sig = nn.Sigmoid()

    def forward(self, context, bert_masks, seq_len):
        # context [batch_size, sen_len]

        # 文本encode，文本cls
        encoder_out, _ = self.bert(context, attention_mask=bert_masks, return_dict=False)  #获取bert输出

        h0 = torch.zeros((self.no_layers*2,context.shape[0],self.hidden_dim)).to(device)    #加入训练的h0信息
        c0 = torch.zeros((self.no_layers*2,context.shape[0],self.hidden_dim)).to(device)   #加入训练的c0信息
#         h0 = torch.zeros((self.no_layers*2,batch_size,self.hidden_dim))
#         c0 = torch.zeros((self.no_layers*2,batch_size,self.hidden_dim))
        hidden = (h0,c0)  #定义hidden
#         print(hidden)
        lstm_out, hidden = self.lstm(encoder_out, hidden)  #获取lstm层输出
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)  #lstm_out维度转换
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)  #进行dropout操作
        sig_out = self.fc(out)  #输入到全连接层
        
        # sigmoid function
#         sig_out = self.sig(out)
#         print(sig_out)
#         sig_out.shape
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)  #sig_out维度转换
#         print(sig_out)
#         sig_out.shape

#         sig_out = sig_out[:, -1] # get last batch of labels

        
        # return last sigmoid output and hidden state
#         return sig_out
        return sig_out  #返回维度

In [23]:
no_layers = 2  #lstm层数
embedding_dim = 768  #embedding维度
output_dim = len(label_dict)  #输出维度（几分类）
hidden_dim = 256  #隐藏层维度

model = BertLSTM(no_layers,hidden_dim,embedding_dim,drop_prob=0.5)  #实例化模型

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 2  #定义batch_size

dataloader_train = DataLoader(  #使用DataLoader封装使模型能进行batch训练
    dataset_train,   #训练集
    sampler=RandomSampler(dataset_train),  #随机采样
    batch_size=batch_size,  #定义batch_size
    drop_last=True  #丢弃不满足一个batch的数据
)

dataloader_val = DataLoader(  #使用DataLoader封装使模型能进行batch训练
    dataset_val,  #测试集
    sampler=RandomSampler(dataset_val),  #随机采样
    batch_size=batch_size,  #定义batch_size
    drop_last=True  #丢弃不满足一个batch的数据
)

In [25]:
from transformers import AdamW, get_linear_schedule_with_warmup

criterion = nn.CrossEntropyLoss()  #定义loss函数
optimizer = torch.optim.AdamW(   #定义优化器
    model.parameters(),
    lr = 1e-5,  #学习率
    eps = 1e-8  #AdamW系数
)

In [26]:
epochs = 8  #定义训练epochs

scheduler = get_linear_schedule_with_warmup(  #进行学习率warmup
    optimizer,   #优化器
    num_warmup_steps=0,  #warmup起点为0
    num_training_steps = len(dataloader_train)*epochs #训练轮数
)

In [27]:
import numpy as np
from sklearn.metrics import f1_score

In [28]:
def f1_score_func(preds, labels):  #计算f1_score函数
    preds_flat = np.argmax(preds, axis=1).flatten()  #获取预测label
    labels_flat = labels.flatten()  #获取真实label
    return f1_score(labels_flat, preds_flat, average = 'weighted')  #计算f1_score使用‘weighted’权重

In [29]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [30]:
import random

seed_val = 17  #设置随机数
random.seed(seed_val)   #设置随机种子
np.random.seed(seed_val)  #设置np随机种子
torch.manual_seed(seed_val)  #设置torch随机种子
torch.cuda.manual_seed_all(seed_val)  #设置cuda随机种子

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  #设置训练的设备如有gpu选择gpu没有就cpu
model.to(device)  #将模型放入 device
print(device)  #打印 device

cuda


In [32]:
# 自定义bert
def evaluate(dataloader_val):
#定义评估函数
    model.eval()  #模型做为预测
#     model = model.to(device)
    loss_val_total = 0  #loss_val_total置零
    predictions, true_vals = [], []  #定义预测和真实值
    
    for batch in tqdm(dataloader_val):  #对于每个batch
        
        batch = tuple(b.to(device) for b in batch)  #batch放入device
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }  #获取模型输入

        with torch.no_grad():         #不做梯度求导
            outputs = model(inputs.get('input_ids'), inputs.get('attention_mask'), 256)
          #获取模型输出并裁减索引位置为1的的维度
        loss = criterion(outputs, inputs['labels'])  #计算loss
        logits = outputs  #获取输出
        loss_val_total += loss.item()  #累计loss

        logits = logits.detach().cpu().numpy()   #将logits放回cpu并转成numpy
        label_ids = inputs['labels'].cpu().numpy()  #将labels放回cpu并转成numpy
        predictions.append(logits)  #预测值累加
        true_vals.append(label_ids)  #真实值累加
    
    loss_val_avg = loss_val_total/len(dataloader_val)   #计算平均loss
    
    predictions = np.concatenate(predictions, axis=0)   #合并所有预测值
    true_vals = np.concatenate(true_vals, axis=0)   #合并所有真实值
            
    return loss_val_avg, predictions, true_vals  #返回结果

In [33]:
# 自定义bert
for epoch in tqdm(range(1, epochs+1)):  #对于每一轮
# for epoch in tqdm(range(1, 1)):
    true_vals = []   #定义真实值列表
    pred_label = []  #定义预测值列表
    model.train()  #模型进行训练
#     model = model.to(device)
    loss_train_total = 0  #初始化loss数值
    
    progress_bar = tqdm(dataloader_train,   #定义一个可视化的训练进度条
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:  #对于每个batch
        model.zero_grad()  #清空梯度信息
        batch = tuple(b.to(device) for b in batch)   #batch放入device

        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
#         outputs = model(inputs.get('input_ids'),inputs.get('attention_mask'))
#         outputs = model(inputs.get('input_ids'), inputs.get('attention_mask'), 256).squeeze(0)
        outputs = model(inputs.get('input_ids'), inputs.get('attention_mask'), 256)  #获取模型输出
#         outputs = model(**inputs) # ok的
#         print(outputs)

        loss = criterion(outputs, inputs['labels'])  #计算loss
        loss_train_total +=loss.item()  #累计loss
        loss.backward()  #反向传播
        
        
        
        label_ids = inputs['labels'].cpu().numpy()  #将labels放回cpu并转成numpy
        true_vals.append(label_ids)  #累加标签值
        pred_y = np.argmax(outputs.cpu().detach().numpy(), axis=1).flatten()  #将预测数值放回cpu并转成数组
        pred_label.append(pred_y)
        
        
#         true_vals = true_vals.flatten()
        

         
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  #梯度裁减
        
        optimizer.step()  #优化器迭代
        scheduler.step()  #学习率迭代
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})      #可视化loss数值
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch：{epoch}')  #可视化步数
    
    true_vals = [y for x in true_vals for y in x]  #获取真实值
    pred_label = [y for x in pred_label for y in x]  #获取预测值
    report = classification_report(true_vals, pred_label, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], target_names=['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'])
    print(report)  #采用classification_report输出可视化训练集评估指标并打印
    
    loss_train_avg = loss_train_total/len(dataloader_train)  #计算平均loss
    tqdm.write(f'Training loss: {loss_train_avg}')  #可视化loss
    
#     tqdm.write(f'socre: {report}')
    
    val_loss, predictions, true_vals_val = evaluate(dataloader_val)  #模型验证
    val_f1 = f1_score_func(predictions, true_vals_val)  #计算f1
#     val_acc = accuracy_per_class(predictions, true_vals)

    preds_val = np.argmax(predictions, axis=1).flatten()  #获取预测数值
    report_val = classification_report(true_vals_val, preds_val, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], target_names=['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'])
    print(report_val)  #采用classification_report输出可视化验证集评估指标并打印
    
    tqdm.write(f'Validation loss: {val_loss}')  #可视化loss
    tqdm.write(f'F1 Score (weighted): {val_f1}')   #可视化f1
#     tqdm.write(f'Accuracy (weighted): {val_acc}')

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/22483 [00:00<?, ?it/s]


Epoch：1
              precision    recall  f1-score   support

          体育       0.99      0.95      0.97      4500
          娱乐       0.95      0.94      0.95      4500
          家居       0.94      0.91      0.93      4482
          房产       0.90      0.88      0.89      4498
          教育       0.92      0.88      0.90      4499
          时尚       0.94      0.92      0.93      4493
          时政       0.90      0.88      0.89      4500
          游戏       0.97      0.94      0.95      4496
          科技       0.91      0.90      0.91      4500
          财经       0.92      0.89      0.91      4498

   micro avg       0.93      0.91      0.92     44966
   macro avg       0.93      0.91      0.92     44966
weighted avg       0.93      0.91      0.92     44966

Training loss: 0.6438265247203022


  0%|          | 0/2498 [00:00<?, ?it/s]

              precision    recall  f1-score   support

          体育       0.99      0.99      0.99       499
          娱乐       0.94      0.97      0.96       500
          家居       0.96      0.94      0.95       498
          房产       0.88      0.94      0.91       500
          教育       0.91      0.94      0.92       500
          时尚       0.96      0.96      0.96       499
          时政       0.94      0.90      0.92       500
          游戏       0.98      0.96      0.97       500
          科技       0.96      0.91      0.93       500
          财经       0.93      0.93      0.93       500

    accuracy                           0.94      4996
   macro avg       0.95      0.94      0.94      4996
weighted avg       0.95      0.94      0.94      4996

Validation loss: 0.33289485312097916
F1 Score (weighted): 0.944476165940721


...

              precision    recall  f1-score   support

          体育       1.00      1.00      1.00       500
          娱乐       0.98      0.98      0.98       500
          家居       0.96      0.97      0.97       498
          房产       0.93      0.92      0.92       500
          教育       0.95      0.93      0.94       499
          时尚       0.98      0.97      0.97       499
          时政       0.94      0.92      0.93       500
          游戏       0.99      0.98      0.98       500
          科技       0.93      0.96      0.95       500
          财经       0.94      0.95      0.95       500

    accuracy                           0.96      4996
   macro avg       0.96      0.96      0.96      4996
weighted avg       0.96      0.96      0.96      4996

Validation loss: 0.37718342754218614
F1 Score (weighted): 0.9581301658341261


Epoch 8:   0%|          | 0/22483 [00:00<?, ?it/s]


Epoch：8
              precision    recall  f1-score   support

          体育       1.00      1.00      1.00      4500
          娱乐       1.00      1.00      1.00      4500
          家居       1.00      1.00      1.00      4482
          房产       0.99      0.99      0.99      4498
          教育       1.00      1.00      1.00      4499
          时尚       1.00      1.00      1.00      4493
          时政       1.00      1.00      1.00      4500
          游戏       1.00      1.00      1.00      4496
          科技       1.00      1.00      1.00      4500
          财经       0.99      0.99      0.99      4498

    accuracy                           1.00     44966
   macro avg       1.00      1.00      1.00     44966
weighted avg       1.00      1.00      1.00     44966

Training loss: 0.02263884480130473


  0%|          | 0/2498 [00:00<?, ?it/s]

              precision    recall  f1-score   support

          体育       1.00      1.00      1.00       500
          娱乐       0.97      0.98      0.98       500
          家居       0.96      0.97      0.97       498
          房产       0.92      0.92      0.92       500
          教育       0.95      0.93      0.94       500
          时尚       0.98      0.97      0.97       499
          时政       0.94      0.92      0.93       499
          游戏       0.98      0.98      0.98       500
          科技       0.93      0.96      0.94       500
          财经       0.94      0.94      0.94       500

    accuracy                           0.96      4996
   macro avg       0.96      0.96      0.96      4996
weighted avg       0.96      0.96      0.96      4996

Validation loss: 0.4013894011495482
F1 Score (weighted): 0.9575414363739
